In [1]:
import os
from collections import defaultdict
from Bio import SeqIO
import statistics
import pandas as pd
import numpy as np
import math
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import T5Tokenizer, T5Model
import re
import random
import torch
from npy_append_array import NpyAppendArray
os.chdir("/Users/zhongyuanchen/Desktop/CAFA-5-Protein-Function-Prediction")

In [27]:
#graph = obonet.read_obo("/content/drive/MyDrive/cafa-5-protein-function-prediction/Train/go-basic.obo")

In [2]:
# Read in the GO-terms data
terms_df = pd.read_csv('Train/train_terms.tsv',sep="\t")
terms_df.drop(columns = ["aspect"],inplace = True)
terms_df.set_index("EntryID",inplace = True)
freq_counts = terms_df["term"].value_counts()

In [3]:
# Select num_labels most frequent GO terms and restrict to these prediction
num_labels = 1500
chosen_terms = freq_counts.index[:num_labels]
chosent_terms = set(chosen_terms)
filt = terms_df["term"].isin(chosen_terms)
terms_df = terms_df[filt]
terms = terms_df["term"]
sum(freq_counts.iloc[:num_labels])/sum(freq_counts.iloc)
#num_labels = len(chosen_terms)

0.824170378699083

In [8]:
# Multilabel encoding
terms_mlb = MultiLabelBinarizer()
terms_mlb.fit([terms])

MultiLabelBinarizer()

In [7]:
embedded_sqs = np.load("Eembedding/train_embeds.npy")
entry_ids = np.load("Eembedding/train_ids.npy")
embedded_ids = {entry_id:indx for indx,entry_id in enumerate(entry_ids)}

In [14]:
# Define dictionaries for fast acessing of terms and sequences using their EntryIDs
id_terms = defaultdict(list)
for entry_id,row in terms_df.iterrows():
    id_terms[entry_id].append(row["term"])

# Define a data generator object to generate data for each epoch
# Without generator, the size of all sequences add up to 30 gigabytes
class Data_Generator(tf.keras.utils.Sequence):
    
    def __init__(self,indexs, batch_size):
        self.indexs = indexs
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.indexs) / self.batch_size)
    
    def on_epoch_end(self):
        random.shuffle(self.indexs)
    
    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.indexs))
        batch_x = []
        batch_y = []

        for i in range(low,high):
            entry_id = self.indexs[i]
            anotation = id_terms[entry_id]
            batch_x.append(embedded_sqs[embedded_ids[entry_id]])
            batch_y.append(terms_mlb.transform([anotation])[0])

        return np.array(batch_x), np.array(batch_y)


In [15]:
weights = pd.read_csv("IA.txt",header = None,delimiter='\t')
weights.columns = ["term","weight"]
weight_dict = {row["term"]:row["weight"] for indx,row in weights.iterrows()}
weights = {indx:weight_dict[term] for indx,term in enumerate(terms_mlb.classes_)}

In [35]:
# Create training and testing datas by shuffling and spliting an array of EntryIDs
indexs = list(entry_ids)
random.shuffle(indexs)
train_test_split = int(0.3*len(indexs))
train_indexs = indexs[:train_test_split]
test_indexs = indexs[train_test_split:]

# Define the respected generators for training and testing
train_generator = Data_Generator(train_indexs,512)
test_generator = Data_Generator(test_indexs,512)

# Create a simple CNN for multilabel classification
#model = models.Sequential()
#model.add(layers.Dense(num_labels,activation = 'sigmoid'))

#
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=[1024]),    
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=num_labels,activation='sigmoid')
])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

# Compile and train the model
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['binary_accuracy', tf.keras.metrics.AUC()],
             )
model.fit(x = train_generator,validation_data = test_generator,epochs = 20, callbacks=my_callbacks)

Epoch 1/20
84/84 [==============================] - 9s 99ms/step - loss: 0.1164 - binary_accuracy: 0.9679 - auc_5: 0.7772 - val_loss: 0.5034 - val_binary_accuracy: 0.9803 - val_auc_5: 0.8550
Epoch 2/20
84/84 [==============================] - 8s 100ms/step - loss: 0.0693 - binary_accuracy: 0.9805 - auc_5: 0.8884 - val_loss: 0.4049 - val_binary_accuracy: 0.9807 - val_auc_5: 0.9061
Epoch 3/20
84/84 [==============================] - 8s 100ms/step - loss: 0.0646 - binary_accuracy: 0.9808 - auc_5: 0.9128 - val_loss: 0.3049 - val_binary_accuracy: 0.9809 - val_auc_5: 0.9196
Epoch 4/20
84/84 [==============================] - 8s 98ms/step - loss: 0.0622 - binary_accuracy: 0.9811 - auc_5: 0.9229 - val_loss: 0.2210 - val_binary_accuracy: 0.9810 - val_auc_5: 0.9266
Epoch 5/20
84/84 [==============================] - 8s 101ms/step - loss: 0.0605 - binary_accuracy: 0.9813 - auc_5: 0.9289 - val_loss: 0.1398 - val_binary_accuracy: 0.9811 - val_auc_5: 0.9289
Epoch 6/20
84/84 [========================

In [37]:
model.load_weights("model.11-0.06.h5")
model.evaluate(test_generator)

195/195 [==============================] - 5s 27ms/step - loss: 0.0610 - binary_accuracy: 0.9812 - auc_5: 0.9230


[0.06096853315830231, 0.9811958074569702, 0.9229578971862793]

In [38]:
# Read in test data and preprocess similarly
embedded_sqs_test = np.load("Eembedding/test_embeds.npy")
embedded_ids_test = np.load("Eembedding/test_ids.npy")
prediction = model.predict(embedded_sqs_test)

4434/4434 [==============================] - 6s 1ms/step


In [39]:
with open("submission_test.tsv","w") as f:
    for indx,entry_id in enumerate(embedded_ids_test):
        for i in range(1500):
            print (f"{entry_id}\t{terms_mlb.classes_[i]}\t{prediction[indx][i]}",file = f)
        if indx%3000 == 0:
            print(f"{indx/len(embedded_ids_test)}% completed")
    print("Completed")

0.0% completed
0.02114686497726712% completed
0.04229372995453424% completed
0.06344059493180136% completed
0.08458745990906848% completed
0.1057343248863356% completed
0.12688118986360272% completed
0.14802805484086984% completed
0.16917491981813695% completed
0.19032178479540407% completed
0.2114686497726712% completed
0.23261551474993833% completed
0.25376237972720544% completed
0.2749092447044726% completed
0.2960561096817397% completed
0.3172029746590068% completed
0.3383498396362739% completed
0.35949670461354105% completed
0.38064356959080814% completed
0.4017904345680753% completed
0.4229372995453424% completed
0.4440841645226095% completed
0.46523102949987666% completed
0.48637789447714375% completed
0.5075247594544109% completed
0.528671624431678% completed
0.5498184894089452% completed
0.5709653543862122% completed
0.5921122193634794% completed
0.6132590843407465% completed
0.6344059493180136% completed
0.6555528142952807% completed
0.6766996792725478% completed
0.6978465442

In [31]:
pred_df = []
for indx,entry_id in enumerate(embedded_ids_test):
    for i in range(1500):
        pred_df.append([entry_id,terms_mlb.classes_[i],prediction[indx][i]])
    if indx%3000 == 0:
        print(f"{indx/len(embedded_ids_test)}% completed")
print("Completed")

submission = pd.DataFrame(pred_df)
submission.columns = ["EntryID","Terms","Prediction"]
submission.to_csv('submission.tsv', sep="\t",header = False,index = False)

0.0% completed
0.02114686497726712% completed
0.04229372995453424% completed
0.06344059493180136% completed
0.08458745990906848% completed
0.1057343248863356% completed
0.12688118986360272% completed
0.14802805484086984% completed
0.16917491981813695% completed
0.19032178479540407% completed
0.2114686497726712% completed
0.23261551474993833% completed
0.25376237972720544% completed
0.2749092447044726% completed
0.2960561096817397% completed
0.3172029746590068% completed
0.3383498396362739% completed
0.35949670461354105% completed
0.38064356959080814% completed
0.4017904345680753% completed
0.4229372995453424% completed
0.4440841645226095% completed
0.46523102949987666% completed
0.48637789447714375% completed
0.5075247594544109% completed
0.528671624431678% completed
0.5498184894089452% completed
0.5709653543862122% completed
0.5921122193634794% completed
0.6132590843407465% completed
0.6344059493180136% completed
0.6555528142952807% completed
0.6766996792725478% completed
0.6978465442

In [ ]:
trimed = submission[submission["Prediction"] > 0.4]

In [ ]:
len(trimed)

In [ ]:
trimed.to_csv('submission.tsv', sep="\t",header = False,index = False)

In [14]:
submission.to_csv('submission.tsv', sep="\t",header = False,index = False)

In [15]:
print("\n\n\n\n\n\n\n\n\n\n\n\n\n")